In [6]:
from src.datasets import stellar_data
from src.models import vanilla_stellar
import torch
import numpy as np
import argparse

In [7]:
torch.manual_seed(42)
sample_idx_permutations = []
n_folds = 5
n_train = 100
for i in range(n_folds):
    idx_perm = torch.randperm(125).tolist()
    sample_idx_permutations.append({"train" : idx_perm[:n_train], "valid" : idx_perm[n_train:]})

In [12]:
cfg_reduced = argparse.Namespace(**{
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'input_dim': 41,
    'hid_dim': 512, # originally 128
    'num_classes': 14,
    'lr': 1e-3,
})

In [13]:
n_epochs = 20

In [14]:
valid_accs = np.zeros(n_folds)
for k in range(n_folds):
    print(f"Fold {k} of {n_folds}")
    train_dataloader = stellar_data.StellarDataloader(
        filename='stellar_graph_dataset_train.pt',
        batch_size=512,
        shuffle=True,
        graphs_idx=sample_idx_permutations[k]["train"],
        test=False
    )
    valid_dataloader = stellar_data.StellarDataloader(
        filename='stellar_graph_dataset_train.pt',
        batch_size=512,
        shuffle=False,
        graphs_idx=sample_idx_permutations[k]["valid"],
        test=False
    )

    vanilla_stellar_reduced = vanilla_stellar.VanillaStellarReduced(cfg_reduced)
    valid_acc = vanilla_stellar_reduced.train(
        train_loader=train_dataloader,
        valid_loader=valid_dataloader,
        epochs=n_epochs,
        return_valid_acc=True)
    valid_accs[k] = valid_acc

Fold 0 of 5


Validation - epoch 19: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s, Loss=1.61, Accuracy=0.564]


Fold 1 of 5


Validation - epoch 19: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s, Loss=1.76, Accuracy=0.492]


Fold 2 of 5


Validation - epoch 19: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s, Loss=1.49, Accuracy=0.638]


Fold 3 of 5


Validation - epoch 19: 100%|██████████| 1/1 [00:00<00:00,  1.26it/s, Loss=2.03, Accuracy=0.375]


Fold 4 of 5


Validation - epoch 19: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s, Loss=1.69, Accuracy=0.554]


In [15]:
print(f"Mean validation accuracy: {valid_accs.mean()} +/- {valid_accs.std()}")

Mean validation accuracy: 0.5245523750782013 +/- 0.08812878765431796
